In [1]:
from pathlib import Path
import os
STUDENT = 'mmr497'

OUTLIERS_PATH = Path('./outliers2/')
INTERMEDIATE_PATH = Path('./intermediate_datafiles/')
os.chdir(f'/home/{STUDENT}/')
# from Python3Code.outliers_distance_parquet import outlier_detector


In [2]:
import pandas as pd
import numpy as np
import sys
from util.VisualizeDataset import VisualizeDataset
from Chapter3.OutlierDetection import DistanceBasedOutlierDetection
from Chapter3.ImputationMissingValues import ImputationMissingValues as MisVal
from Python3Code.outlier_detector import OutlierDetector

In [3]:
# intermed_df = pd.read_parquet(f'{INTERMEDIATE_PATH}/ML4QS_combined_results_2.parquet')
#
# sensor_prefixes = [
#     'acc_phone_',
#     'lin_acc_phone_',
#     'gyr_phone_',
#     'mag_phone_',
# ]
#
# folder = 'intermediate_datafiles'
# output_folder = 'outliers2'
# vehicles = ['train', 'bus', 'metro', 'tram', 'car','walking']
#
# files = [f for f in os.listdir(folder) if f.endswith('.parquet')]
# data_by_vehicle = {vehicle: [] for vehicle in vehicles}
#
# for vehicle in vehicles:
#     data_by_vehicle[vehicle] = intermed_df[~intermed_df[f'label{vehicle}'].isna()]
#
# for instance_id in intermed_df['id'].unique():
#     # Create a mask for the current instance
#     instance_mask = intermed_df['id'] == instance_id
#     ins_df = intermed_df[instance_mask]
#
#     start_time = ins_df['timestamp'].iloc[0]
#     end_time = ins_df['timestamp'].iloc[-1]
#
#     print(start_time)
#
#     start_time_plus_10s = start_time + pd.Timedelta(seconds=10)
#     end_time_minus_10s = end_time - pd.Timedelta(seconds=10)
#
#     time_mask = (intermed_df.timestamp >= start_time_plus_10s) & (intermed_df.timestamp <= end_time_minus_10s)
#
#     intermed_df = intermed_df[~instance_mask | (instance_mask & time_mask)]
#
# outlier_detector = DistanceBasedOutlierDetection()
#
# cols_to_check_acc = ['acc_phone_X', 'acc_phone_Y', 'acc_phone_Z']
# cols_to_check_lin_acc = ['lin_acc_phone_X', 'lin_acc_phone_Y', 'lin_acc_phone_Z']
# cols_to_check_gyr = ['gyr_phone_X', 'gyr_phone_Y', 'gyr_phone_Z']
# cols_to_check_mag = ['mag_phone_X', 'mag_phone_Y', 'mag_phone_Z']
# cols_to_check_accuracy = ['location_phone_Horizontal Accuracy', 'location_phone_Vertical Accuracy']
#
# d_function = 'euclidean'
# vehicle_params = {
#     'train': {'dmin': 0.15, 'fmin': 0.1},
#     'bus': {'dmin': 0.5, 'fmin': 0.1},
#     'metro': {'dmin': 0.4, 'fmin': 0.1},
#     'tram': {'dmin': 0.45, 'fmin': 0.1},
#     'car': {'dmin': 0.1, 'fmin': 0.05},
#     'walking': {'dmin': 0.65, 'fmin': 0.1},
# }
#
# Path(output_folder).mkdir(parents=True, exist_ok=True)
#
# data_by_vehicle_buffer = {vehicle: [] for vehicle in vehicles}
#
# for vehicle in vehicles:
#     data_by_vehicle_buffer[vehicle] = intermed_df[~intermed_df[f'label{vehicle}'].isna()]
#
# for vehicle, df in data_by_vehicle.items():
#     raw_output_file = os.path.join(output_folder, f"{vehicle}_raw.parquet")
#     df.reset_index().to_parquet(raw_output_file, version='2.6', allow_truncated_timestamps=True)
#     print(f"Saved raw {vehicle} data to {raw_output_file}")
#
#
# for vehicle, df in data_by_vehicle_buffer.items():
#     print(f"Processing {vehicle} data...")
#
#     vehicle_dmin = vehicle_params[vehicle]['dmin']
#     vehicle_fmin = vehicle_params[vehicle]['fmin']
#
#     # TODO: This is too extreme and marks nearly everything as an outlier.
#
#     df = outlier_detector.simple_distance_based(df, cols=cols_to_check_acc, d_function=d_function, dmin=vehicle_dmin, fmin=vehicle_fmin)
#     df.rename(columns={'simple_dist_outlier': f'simple_dist_outlier_acc'}, inplace=True)
#     df = outlier_detector.simple_distance_based(df, cols=cols_to_check_lin_acc, d_function=d_function, dmin=vehicle_dmin, fmin=vehicle_fmin)
#     df.rename(columns={'simple_dist_outlier': f'simple_dist_outlier_lin_acc'}, inplace=True)
#     df = outlier_detector.simple_distance_based(df, cols=cols_to_check_gyr, d_function=d_function, dmin=vehicle_dmin, fmin=vehicle_fmin)
#     df.rename(columns={'simple_dist_outlier': f'simple_dist_outlier_gyr'}, inplace=True)
#     df = outlier_detector.simple_distance_based(df, cols=cols_to_check_mag, d_function=d_function, dmin=vehicle_dmin, fmin=vehicle_fmin)
#     df.rename(columns={'simple_dist_outlier': f'simple_dist_outlier_mag'}, inplace=True)
#
#     df_filtered = df
#
# ##################################################
# ##################################################
#
#     output_file = os.path.join(output_folder, f"{vehicle}_filtered_norem.parquet")
#     df_filtered.reset_index().to_parquet(output_file, version='2.6', allow_truncated_timestamps=True)
#     print(f"Saved filtered {vehicle} data to {output_file}")

In [4]:
intermed_df = pd.read_parquet(f'{INTERMEDIATE_PATH}/ML4QS_combined_results_2.parquet')
out_detector = OutlierDetector(intermed_df)
intermed_df = out_detector.fit_transform("chauvenet", C=2)

In [5]:
# dataframes = []
# for filtered_frame in OUTLIERS_PATH.glob('*_filtered_norem.parquet'):
#     print(f'processing {filtered_frame}...')
#     pd.read_parquet(filtered_frame)
#     dataframes.append(pd.read_parquet(filtered_frame))
#
# full_df = pd.concat(dataframes).set_index('index')
# full_df.head()

# START IMPUTATION

In [6]:
time_gap_threshold = 250

def time_gap(group):
    time_series = pd.to_datetime(group['timestamp'])
    time_gap_dfs = time_series.diff()
    threshold = pd.Timedelta(milliseconds=time_gap_threshold)
    time_gaps = (time_gap_dfs > threshold).sum()
    return time_gaps



In [7]:
# full_df[full_df['id'] == 0]

In [8]:
imputation_df = intermed_df.copy()

# cols_to_check_acc = ['acc_phone_X', 'acc_phone_Y', 'acc_phone_Z']
# cols_to_check_lin_acc = ['lin_acc_phone_X', 'lin_acc_phone_Y', 'lin_acc_phone_Z']
# cols_to_check_gyr = ['gyr_phone_X', 'gyr_phone_Y', 'gyr_phone_Z']
# cols_to_check_mag = ['mag_phone_X', 'mag_phone_Y', 'mag_phone_Z']
#
# checking = {'simple_dist_outlier_acc': cols_to_check_acc,
#             'simple_dist_outlier_lin_acc': cols_to_check_lin_acc,
#             'simple_dist_outlier_gyr': cols_to_check_gyr,
#             'simple_dist_outlier_mag': cols_to_check_mag}
#
bad_sensors = ['proximity', 'location']
drop_cols = []
for sensor in bad_sensors:
    drop_cols.extend(imputation_df.columns[imputation_df.columns.str.contains(sensor)])
#
imputation_df.drop(drop_cols, axis=1, inplace=True)
#
# columns_to_convert = [col for col in imputation_df.columns
#                      if col not in ['id', 'timestamp', 'simple_dist_outlier']
#                      and 'label' not in col]
#
# df_with_nans = imputation_df.copy()
#
# for outlier_col, cols_to_check in checking.items():
#     df_with_nans.loc[imputation_df[outlier_col], cols_to_check] = np.nan
#
# imputation_df = df_with_nans
#
# imputation_df.isna().sum()

# gaps_per_instance = imputation_df.groupby('id').apply(time_gap)
# total_gaps = gaps_per_instance
#
# total_gaps

In [9]:
imputer = MisVal()
cols_to_impute = [col for col in imputation_df.columns if col not in ['id', 'timestamp'] and 'label' not in col and 'simple_dist_outlier' not in col]

print("NaN values before imputation:")
print(imputation_df[cols_to_impute].isna().sum())

for instance in imputation_df.id.unique():
    instance_mask = imputation_df.id == instance
    for col in cols_to_impute:
        if imputation_df.loc[instance_mask, col].isna().any():
            print(f"Imputing {col} for instance {instance}...")
            print(imputation_df.loc[instance_mask, col].isna().sum() / len(imputation_df.loc[instance_mask, col]))
            imputation_df[instance_mask] = imputer.impute_interpolate(dataset=imputation_df[instance_mask].copy(), col=col)

NaN values before imputation:
acc_phone_X                  2
acc_phone_Y                  0
acc_phone_Z                  0
lin_acc_phone_X            263
lin_acc_phone_Y            358
lin_acc_phone_Z            382
gyr_phone_X                331
gyr_phone_Y                150
gyr_phone_Z                116
mag_phone_X                205
mag_phone_Y                164
mag_phone_Z                130
id_outlier                   0
acc_phone_X_outlier          0
acc_phone_Y_outlier          0
acc_phone_Z_outlier          0
lin_acc_phone_X_outlier      0
lin_acc_phone_Y_outlier      0
lin_acc_phone_Z_outlier      0
gyr_phone_X_outlier          0
gyr_phone_Y_outlier          0
gyr_phone_Z_outlier          0
mag_phone_X_outlier          0
mag_phone_Y_outlier          0
mag_phone_Z_outlier          0
dtype: int64
Imputing lin_acc_phone_X for instance 0.0...
0.0009478672985781991
Imputing lin_acc_phone_Y for instance 0.0...
0.0018957345971563982
Imputing lin_acc_phone_Z for instance 0.0...
0.0

In [13]:
visualise_df = imputation_df.copy()

In [14]:
outlier_cols = [col for col in visualise_df.columns if 'outlier' in col]
visualise_df = visualise_df.drop(outlier_cols, axis=1)
visualise_df.columns

Index(['id', 'timestamp', 'acc_phone_X', 'acc_phone_Y', 'acc_phone_Z',
       'lin_acc_phone_X', 'lin_acc_phone_Y', 'lin_acc_phone_Z', 'gyr_phone_X',
       'gyr_phone_Y', 'gyr_phone_Z', 'mag_phone_X', 'mag_phone_Y',
       'mag_phone_Z', 'labelbus', 'labeltram', 'labelmetro', 'labelcar',
       'labelwalking', 'labeltrain'],
      dtype='object')

In [15]:
notebook_path = 'imputation.ipynb'

label_fix = True

if 'timestamp' in visualise_df.columns:
    visualise_df.set_index('timestamp', inplace=True)

if label_fix:
    # This is very stupid
    label_col = [col for col in visualise_df.columns if 'label' in col]
    for col in label_col:
        visualise_df[col] = visualise_df[col].apply(lambda x: 1 if x == 0 or x == 1 else 0)

DataViz = VisualizeDataset(notebook_path)

# TODO: This should work nicely per instance once the outlier problem is fixed.

# Plot all data
for instance in imputation_df.id.unique():
    visual_instance = visualise_df[visualise_df.id == instance]
    DataViz.plot_dataset(visual_instance,
                        ['acc_', 'gyr_', 'lin_' , 'mag_', 'label'],
                        ['like', 'like', 'like', 'like', 'like', 'like'],
                        ['line', 'line', 'line', 'line', 'line', 'points'])

Figure saved to figures/imputation/figure_1.png
Figure saved to figures/imputation/figure_1.pdf
Figure saved to figures/imputation/figure_2.png
Figure saved to figures/imputation/figure_2.pdf
Figure saved to figures/imputation/figure_3.png
Figure saved to figures/imputation/figure_3.pdf
Figure saved to figures/imputation/figure_4.png
Figure saved to figures/imputation/figure_4.pdf
Figure saved to figures/imputation/figure_5.png
Figure saved to figures/imputation/figure_5.pdf
Figure saved to figures/imputation/figure_6.png
Figure saved to figures/imputation/figure_6.pdf


/home/mmr497/util/VisualizeDataset.py:58: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  f, xar = plt.subplots(len(columns), sharex=True, sharey=False)


Figure saved to figures/imputation/figure_7.png
Figure saved to figures/imputation/figure_7.pdf


/home/mmr497/util/VisualizeDataset.py:58: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  f, xar = plt.subplots(len(columns), sharex=True, sharey=False)


Figure saved to figures/imputation/figure_8.png
Figure saved to figures/imputation/figure_8.pdf


...

Figure saved to figures/imputation/figure_12.png
Figure saved to figures/imputation/figure_12.pdf


/home/mmr497/util/VisualizeDataset.py:58: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  f, xar = plt.subplots(len(columns), sharex=True, sharey=False)


Figure saved to figures/imputation/figure_13.png
Figure saved to figures/imputation/figure_13.pdf


/home/mmr497/util/VisualizeDataset.py:58: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  f, xar = plt.subplots(len(columns), sharex=True, sharey=False)


Figure saved to figures/imputation/figure_14.png
Figure saved to figures/imputation/figure_14.pdf


In [19]:
for idx in imputation_df.index:
    if not pd.to_datetime(idx):
        print(idx)
        break

In [21]:
notebook_path = 'imputation2.ipynb'

DataViz = VisualizeDataset(notebook_path)

day_slices = visualise_df.groupby(visualise_df.index.date)

for day, day_slice in day_slices:
    DataViz.plot_dataset(day_slice,
                            ['acc_', 'gyr_', 'lin_' , 'mag_', 'label'],
                            ['like', 'like', 'like', 'like', 'like'],
                            ['line', 'line', 'line', 'line', 'points'])

/home/mmr497/util/VisualizeDataset.py:58: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  f, xar = plt.subplots(len(columns), sharex=True, sharey=False)


Figure saved to figures/imputation2/figure_1.png
Figure saved to figures/imputation2/figure_1.pdf


/home/mmr497/util/VisualizeDataset.py:58: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  f, xar = plt.subplots(len(columns), sharex=True, sharey=False)


Figure saved to figures/imputation2/figure_2.png
Figure saved to figures/imputation2/figure_2.pdf


/home/mmr497/util/VisualizeDataset.py:58: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  f, xar = plt.subplots(len(columns), sharex=True, sharey=False)


Figure saved to figures/imputation2/figure_3.png
Figure saved to figures/imputation2/figure_3.pdf


/home/mmr497/util/VisualizeDataset.py:58: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  f, xar = plt.subplots(len(columns), sharex=True, sharey=False)


Figure saved to figures/imputation2/figure_4.png
Figure saved to figures/imputation2/figure_4.pdf
